anyth

In [2]:
!pip install tiktoken

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 222.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.0 kB 187.9 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 203.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/799.0 kB ? eta -:--:--
    --------------------------------------- 10.2/799.0 kB ? eta -:--:--
   - ------------------------------------- 41.0/799.0 kB 653.6 kB/s eta 0:00:02
   -- ------------------------------------ 61.4/799.0 kB 465.5 kB/s eta 0:00:02
   ----- -------------------------------- 122.9/799.0 kB 717.5 kB/s eta 0:00:01
   ------- ------------------------------ 153.6/799.0 kB 833.5 kB/s eta 0:00:01
   ---------- --------------------------- 225.3/799.0 kB 860.2 kB/s eta 0:00:01



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pythonw.exe -m pip install --upgrade pip


In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [2]:
torch.zeros((1,1), device='cuda')

tensor([[0.]], device='cuda:0')

In [12]:
!pip install tensorflow && pip install tf

In [1]:
#!/usr/bin/env python

# This code is adapted from this source 
# https://github.com/idiap/fast-transformers

"""Setup fast transformers"""

from functools import lru_cache
from itertools import dropwhile
from os import path
from setuptools import find_packages, setup
from subprocess import DEVNULL, call

try:
    import torch
    from torch.utils.cpp_extension import BuildExtension, CppExtension
except ImportError as e:
    raise ImportError(
        ("PyTorch is required to install pytorch-fast-transformers. Please "
         "install your favorite version of PyTorch, we support 1.3.1, 1.5.0 "
         "and >=1.6"),
        name=e.name,
        path=e.path
    ) from e


@lru_cache(None)
def cuda_toolkit_available():
    try:
        call(["nvcc"], stdout=DEVNULL, stderr=DEVNULL)
        return True
    except FileNotFoundError:
        return False


def collect_docstring(lines):
    """Return document docstring if it exists"""
    lines = dropwhile(lambda x: not x.startswith('"""'), lines)
    doc = ""
    for line in lines:
        doc += line
        if doc.endswith('"""\n'):
            break

    return doc[3:-4].replace("\r", "").replace("\n", " ")


def collect_metadata():
    meta = {}
    with open(path.join("fast_transformers", "__init__.py")) as f:
        lines = iter(f)
        meta["description"] = collect_docstring(lines)
        for line in lines:
            if line.startswith("__"):
                key, value = map(lambda x: x.strip(), line.split("="))
                meta[key[2:-2]] = value[1:-1]

    return meta


def get_extensions():
    extensions = [
        CppExtension(
            "fast_transformers.hashing.hash_cpu",
            sources=[
                "fast_transformers/hashing/hash_cpu.cpp"
            ],
            extra_compile_args=["-fopenmp", "-ffast-math"]
        ),
        CppExtension(
            "fast_transformers.aggregate.aggregate_cpu",
            sources=[
               "fast_transformers/aggregate/aggregate_cpu.cpp"
            ],
            extra_compile_args=["-fopenmp", "-ffast-math"]
        ),
        CppExtension(
            "fast_transformers.clustering.hamming.cluster_cpu",
            sources=[
               "fast_transformers/clustering/hamming/cluster_cpu.cpp"
            ],
            extra_compile_args=["-fopenmp", "-ffast-math"]
        ),
        CppExtension(
            "fast_transformers.sparse_product.sparse_product_cpu",
            sources=[
                "fast_transformers/sparse_product/sparse_product_cpu.cpp"
            ],
            extra_compile_args=["-fopenmp", "-ffast-math"]
        ),
        CppExtension(
            "fast_transformers.sparse_product.clustered_sparse_product_cpu",
            sources=[
                "fast_transformers/sparse_product/clustered_sparse_product_cpu.cpp"
            ],
            extra_compile_args=["-fopenmp", "-ffast-math"]
        ),
        CppExtension(
            "fast_transformers.causal_product.causal_product_cpu",
            sources=[
                "fast_transformers/causal_product/causal_product_cpu.cpp"
            ],
            extra_compile_args=["-fopenmp", "-ffast-math"]
        ),
        CppExtension(
            "fast_transformers.local_product.local_product_cpu",
            sources=[
                "fast_transformers/local_product/local_product_cpu.cpp"
            ],
            extra_compile_args=["-fopenmp", "-ffast-math"]
        )
    ]
    if cuda_toolkit_available():
        from torch.utils.cpp_extension import CUDAExtension
        extensions += [
            CUDAExtension(
                "fast_transformers.hashing.hash_cuda",
                sources=[
                    "fast_transformers/hashing/hash_cuda.cu",
                ],
                extra_compile_args=["-arch=compute_50"]
            ),
            CUDAExtension(
                "fast_transformers.aggregate.aggregate_cuda",
                sources=[
                    "fast_transformers/aggregate/aggregate_cuda.cu"
                ],
                extra_compile_args=["-arch=compute_50"]
            ),
            CUDAExtension(
                "fast_transformers.aggregate.clustered_aggregate_cuda",
                sources=[
                    "fast_transformers/aggregate/clustered_aggregate_cuda.cu"
                ],
                extra_compile_args=["-arch=compute_50"]
            ),
            CUDAExtension(
                "fast_transformers.clustering.hamming.cluster_cuda",
                sources=[
                    "fast_transformers/clustering/hamming/cluster_cuda.cu"
                ],
                extra_compile_args=["-arch=compute_50"]
            ),
            CUDAExtension(
                "fast_transformers.sparse_product.sparse_product_cuda",
                sources=[
                    "fast_transformers/sparse_product/sparse_product_cuda.cu"
                ],
                extra_compile_args=["-arch=compute_50"]
            ),
            CUDAExtension(
                "fast_transformers.sparse_product.clustered_sparse_product_cuda",
                sources=[
                    "fast_transformers/sparse_product/clustered_sparse_product_cuda.cu"
                ],
                extra_compile_args=["-arch=compute_50"]
            ),
            CUDAExtension(
                "fast_transformers.causal_product.causal_product_cuda",
                sources=[
                    "fast_transformers/causal_product/causal_product_cuda.cu"
                ],
                extra_compile_args=["-arch=compute_50"]
            ),
            CUDAExtension(
                "fast_transformers.local_product.local_product_cuda",
                sources=[
                    "fast_transformers/local_product/local_product_cuda.cu"
                ],
                extra_compile_args=["-arch=compute_50"]
            )
        ]
    return extensions


def setup_package():
    with open("README.rst") as f:
        long_description = f.read()
    meta = collect_metadata()
    setup(
        name="pytorch-fast-transformers",
        version=meta["version"],
        description=meta["description"],
        long_description=long_description,
        long_description_content_type="text/x-rst",
        maintainer=meta["maintainer"],
        maintainer_email=meta["email"],
        url=meta["url"],
        license=meta["license"],
        classifiers=[
            "Intended Audience :: Science/Research",
            "Intended Audience :: Developers",
            "License :: OSI Approved :: MIT License",
            "Topic :: Scientific/Engineering",
            "Programming Language :: Python",
            "Programming Language :: Python :: 3",
            "Programming Language :: Python :: 3.6",
        ],
        packages=find_packages(exclude=["docs", "tests", "scripts", "examples"]),
        ext_modules=get_extensions(),
        cmdclass={"build_ext": BuildExtension},
        install_requires=["torch"]
    )


if __name__ == "__main__":
    setup_package()

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [4]:
import os 
import math 
import torch 
from sys import argv
from time import time
import numpy as np 
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.tensorboard import SummaryWriter

from text.data import IMBDbByteDataset
from text.model import ByteTextClassifier
from tqdm import tqdm

if len(argv)==2:
    argv.append('1000')
LOG_PATH  = './log/IMDB-4K/mem/'+argv[1]+"_"+argv[2]+"/"

EPOCHS = 26
LOG_FREQ = 100
BATCH_SIZE = 8
WARMUP_STEPS = 4000
LEARNING_RATE = 5e-2

def get_learning_rate_scheduler(optimizer, num_warmup_steps, last_epoch=-1): 
    """
    Create learning rate scheduler. 

    Arguments:
    ----------
        optimizer (:class:`~torch.optim.Optimizer`):
            The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
            The number of steps for the warmup phase.
        last_epoch (:obj:`int`, `optional`, defaults to -1):
            The index of the last epoch when resuming training.

    Returns:
    --------
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """
    def lr_lambda(current_step: int): 
        r = 1.0 

        # Warmup phase
        r *= min(1.0, float(current_step) / float(num_warmup_steps))

        # Decay phase
        r /= math.sqrt(max(current_step, num_warmup_steps))

        return r

    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch)

def evaluate(model, data_loader, criterion, device): 
    # Turn on the evaluation mode
    model.eval() 

    total_loss = 0.0 

    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader): 
            # Unpack batch 
            y = batch['label'].to(device)
            x = batch['tokens'].to(device)
            length_mask = batch['length'].to(device)

            # Call model 
            logits = model(x, length_mask)

            # Accumulate loss 
            total_loss += len(y) * criterion(logits, y).item()

    return total_loss / len(data_loader.dataset)

def predict(model, data_loader, device): 
    # Turn on the evaluation mode
    model.eval() 

    # Initialize outputs
    labels = np.array([])
    predictions = np.array([])
     
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader): 
            # Unpack batch 
            y = batch['label'].to(device)
            x = batch['tokens'].to(device)
            length_mask = batch['length'].to(device)

            # Call model 
            logits = model(x, length_mask)
            probs = nn.functional.softmax(logits, dim=1)
            y_hat = torch.argmax(probs, dim=1)

            labels = np.concatenate((labels, \
                                    y.detach().cpu().numpy()))

            predictions = np.concatenate((predictions, \
                                         y_hat.detach().cpu().numpy()))

    return labels, predictions

if __name__ == '__main__':
    # Specify GPU option 
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Specify model parameters
    params = {
        'n_classes': 2, 
        'classifier_dim': 1024, 
        'max_len': 500, 
        'd_model': 256, 
        'attention_type': 'softmax', 
        'n_layers': 4, 
        'n_heads' : 4,
        'd_ff': 1024, 
        'd_query': 64, 
        'd_values': 64,
        'activation': 'gelu', 
        'output_norm': True
    }

    # Load datasets 
    train_dataset = IMBDbByteDataset(
        training_set=True, 
        sequence_len=params['max_len']
    )

    valid_dataset = IMBDbByteDataset(
        training_set=False, 
        sequence_len=params['max_len']
    )

    # Dataloaders 
    train_loader = DataLoader(
        train_dataset, 
        BATCH_SIZE, 
        shuffle=True, 
        drop_last=False
    )
    valid_loader = DataLoader(
        valid_dataset, 
        BATCH_SIZE, 
        shuffle=False, 
        drop_last=False
    )

    # Model 
    model = ByteTextClassifier(
        vocab_size=train_dataset.vocab_size, 
        **params
    )
    model = torch.nn.DataParallel(model)

    # Set precision 
    model.float()

    # Send to GPU 
    model.to(device)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer 
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=LEARNING_RATE, 
        betas=(0.9, 0.98),
        eps=1e-9, 
        weight_decay=1e-1
    )

    # Scheduler 
    scheduler = get_learning_rate_scheduler(
        optimizer, 
        WARMUP_STEPS
    )

    # Tensorboard
    writer = SummaryWriter(LOG_PATH)
    best_valid_loss = float('inf')    
    best_valid_acc = 0
    # Repeat for each epoch 
    for epoch in tqdm(range(EPOCHS)): 
        # Turn on the training mode
        model.train()

        # Loop over batches 
        for train_batch_idx, train_batch in enumerate(train_loader):
            st=time()
            # Unpack batch 
            y = train_batch['label'].to(device)
            x = train_batch['tokens'].to(device)
            length_mask = train_batch['length'].to(device)

            # Reset graph 
            optimizer.zero_grad()

            # Call model 
            logits = model(x, length_mask)

            # Loss 
            loss = criterion(logits, y)
            loss.backward()

            # Optimization step 
            optimizer.step()

            # Update learning rate 
            scheduler.step()
            et=time()
            #print(et-st)
            # Report results
            if train_batch_idx>0 and train_batch_idx % LOG_FREQ == 0:  
                # Learning rate 
                writer.add_scalar(
                    'Learning Rate', 
                    scheduler.get_last_lr()[0],
                    scheduler._step_count
                )

                # Training loss
                writer.add_scalar(
                    'Training Loss', 
                    loss.item(), 
                    scheduler._step_count
                )

                # Training accuracy 
                y_pred = torch.argmax(
                    nn.functional.softmax(logits, dim=1), 
                    dim=1
                )

                writer.add_scalar(
                    'Training Accuracy', 
                    100*accuracy_score(
                        y.detach().cpu().numpy(), 
                        y_pred.detach().cpu().numpy()
                    ), 
                    scheduler._step_count
                )

                # Validation loss
                valid_loss = evaluate(
                    model, 
                    valid_loader, 
                    criterion, 
                    device
                )
                writer.add_scalar(
                    'Validation Loss', 
                    valid_loss,
                    scheduler._step_count
                )

                # Validation accuracy 
                y_valid, y_valid_pred = predict(
                    model, 
                    valid_loader, 
                    device
                )
                writer.add_scalar(
                    'Validation Accuracy', 
                    100*accuracy_score(y_valid, y_valid_pred), 
                    scheduler._step_count
                )
                valid_accuracy=100*accuracy_score(y_valid, y_valid_pred)
                print(valid_accuracy,100*accuracy_score(
                        y.detach().cpu().numpy(), 
                        y_pred.detach().cpu().numpy()
                    ) )
                if valid_accuracy > best_valid_acc:
                    best_valid_acc = valid_accuracy
                    # Save new model 
                    torch.save(
                        model.state_dict(), 
                        os.path.join(LOG_PATH, 'accuracy-best.pt')
                    )

                # Reset training mode 
                model.train()
        # Evaluate the model 
        valid_loss = evaluate(
            model, 
            valid_loader, 
            criterion, 
            device
        )
        y_valid, y_valid_pred = predict(
            model, 
            valid_loader, 
            device
        )
        valid_accuracy = 100*accuracy_score(y_valid, y_valid_pred)

        # Save best validation loss model 
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            # Save new model 
            torch.save(
                model.state_dict(), 
                os.path.join(LOG_PATH, 'loss-best.pt')
            )

        # Save best validation accuracy model 
        if valid_accuracy > best_valid_acc:
            best_valid_acc = valid_accuracy
            # Save new model 
            torch.save(
                model.state_dict(), 
                os.path.join(LOG_PATH, 'accuracy-best.pt')
            )

    writer.close()

    # Save final model 
    torch.save(
        model.state_dict(), 
        os.path.join(LOG_PATH, 'final.pt')
    )

    memory_stats = torch.cuda.memory_stats()
    max_memory_reserved = torch.cuda.max_memory_reserved(device=device)
    max_memory_allocated = torch.cuda.max_memory_allocated(device=device)

    print('=' * 45, ' Final Checkpoint ', '=' * 45)

    # Training set 
    train_loss = evaluate(model, train_loader, criterion, device)
    y_train, y_train_pred = predict(model, train_loader, device)
    C_train, C_train_counts = np.unique(y_train, return_counts=True)

    # Validation set 
    valid_loss = evaluate(model, valid_loader, criterion, device)
    y_valid, y_valid_pred = predict(model, valid_loader, device)
    C_valid, C_valid_counts = np.unique(y_valid, return_counts=True)

    print('| Training Loss {:5.4f} | Training Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        train_loss, 100*accuracy_score(y_train, y_train_pred), 
        100*f1_score(y_train, y_train_pred, average='micro')))

    print('| Validation Loss {:5.4f} | Validation Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        valid_loss, 100*accuracy_score(y_valid, y_valid_pred), 
        100*f1_score(y_valid, y_valid_pred, average='micro')))

    print('=' * 45, ' Loss Checkpoint ', '=' * 45)

    # Load checkpoint
    model.load_state_dict(torch.load(os.path.join(LOG_PATH, 'loss-best.pt')))

    # Training set 
    train_loss = evaluate(model, train_loader, criterion, device)
    y_train, y_train_pred = predict(model, train_loader, device)
    C_train, C_train_counts = np.unique(y_train, return_counts=True)

    # Validation set 
    valid_loss = evaluate(model, valid_loader, criterion, device)
    y_valid, y_valid_pred = predict(model, valid_loader, device)
    C_valid, C_valid_counts = np.unique(y_valid, return_counts=True)

    print('| Training Loss {:5.4f} | Training Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        train_loss, 100*accuracy_score(y_train, y_train_pred), 
        100*f1_score(y_train, y_train_pred, average='micro')))

    print('| Validation Loss {:5.4f} | Validation Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        valid_loss, 100*accuracy_score(y_valid, y_valid_pred), 
        100*f1_score(y_valid, y_valid_pred, average='micro')))

    print('=' * 45, ' Accuracy Checkpoint ', '=' * 45)

    # Load checkpoint
    model.load_state_dict(torch.load(os.path.join(LOG_PATH, 'accuracy-best.pt')))

    # Training set 
    train_loss = evaluate(model, train_loader, criterion, device)
    y_train, y_train_pred = predict(model, train_loader, device)
    C_train, C_train_counts = np.unique(y_train, return_counts=True)

    # Validation set 
    valid_loss = evaluate(model, valid_loader, criterion, device)
    y_valid, y_valid_pred = predict(model, valid_loader, device)
    C_valid, C_valid_counts = np.unique(y_valid, return_counts=True)

    print('| Training Loss {:5.4f} | Training Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        train_loss, 100*accuracy_score(y_train, y_train_pred), 
        100*f1_score(y_train, y_train_pred, average='micro')))

    print('| Validation Loss {:5.4f} | Validation Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        valid_loss, 100*accuracy_score(y_valid, y_valid_pred), 
        100*f1_score(y_valid, y_valid_pred, average='micro')))

    print('=' * 45, ' Memory Statistics ', '=' * 45)

    print('Max Memory Reserved: ', max_memory_reserved)
    print('Max Memory Allocated: ', max_memory_allocated)

    peak_bytes_requirement = memory_stats["allocated_bytes.all.peak"]

    print('Peak Allocated Bytes: ', peak_bytes_requirement)


No config specified, defaulting to config: imdb_reviews/plain_text
Load dataset info from C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0
Reusing dataset imdb_reviews (C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0)
Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.
Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.
Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.
Constructing tf.data.Dataset imdb_reviews for split None, from C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0
No config specified, defaulting to config: imdb_reviews/plain_text
Load dataset info from C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0
Reusing dataset imdb_reviews (C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0)
Creatin

InvalidArgumentError: Failed to create a directory: ./log/IMDB-4K/mem/--f=c:; Invalid argument

In [5]:
!pip install Cython

  Using cached Cython-3.0.10-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
Using cached Cython-3.0.10-cp311-cp311-win_amd64.whl (2.8 MB)


In [6]:
from setuptools import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("setup.py"))

# %cd ../hadamard_cuda  
# !python setup.py install

# %cd ../diag_mult_cuda 
# !python setup.py install

Compiling setup.py because it changed.
[1/1] Cythonizing setup.py


c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: c:\Users\HP\Documents\hard\D.I\CSC 1401\OnLearningTheKernelmain\setup.py
  tree = Parsing.p_module(s, pxd, full_module_name)


AttributeError: 'tuple' object has no attribute 'tb_frame'